In [ ]:
import os
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from pororo import Pororo

In [ ]:
data_dir = os.getcwd() + '/dataset/korean_unsmile_dataset/'
train_data_file = 'unsmile_train_v1.0.tsv'
test_data_file = 'unsmile_valid_v1.0.tsv'
vectorizer = Pororo(task='sentence_embedding', lang='ko')

In [ ]:
train_df = pd.read_csv(data_dir + train_data_file, sep='\t')
test_df = pd.read_csv(data_dir + test_data_file, sep='\t')
print(len(train_df), len(test_df))

In [ ]:
merged_df = pd.concat([train_df, test_df], axis=0, ignore_index=True)
print(len(merged_df))
len(train_df) + len(test_df) == len(merged_df)

In [ ]:
# 결측치 제거
indexes = []
for i in range(len(merged_df)):
    data = merged_df.iloc[i]
    if data[1:].sum() == 0:
        indexes.append(i)
merged_df.iloc[indexes]

In [ ]:
merged_df.drop(indexes, inplace=True)
merged_df.reset_index(inplace=True)
print(len(merged_df))
merged_df.tail()